In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

Loading the data set:

In [0]:
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()


building classes and scaling the data to fit values between 0 and 1.

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
x_train = x_train/ 255
x_test = x_test / 255

After all of this, the actual models are generated using different activation functions:

In [0]:
 #ELU, Leaky ReLU, PReLU, SoftPlus, Sigmoid, etc.
actF = ['relu', 'elu', 'softplus', 'sigmoid', 'LeakyReLU', 'PReLU'] 
models = {}
tf.keras.layers.LeakyReLU(alpha=0.3)
tf.keras.layers.PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=None)
tf.keras.layers.ELU(alpha=1.0)

for func in range(0,6):
  models[func] = tf.keras.Sequential()
  models[func].add(tf.keras.layers.Flatten(input_shape=(28,28)))
  if func < 4:
    models[func].add(tf.keras.layers.Dense(128,activation=actF[func]))
  else:
    if actF[func] == 'LeakyReLU':
      models[func].add(tf.keras.layers.Dense(128))
      models[func].add(tf.keras.layers.LeakyReLU(alpha=0.05))
    if actF[func] == 'PReLU':
      models[func].add(tf.keras.layers.Dense(128))
      models[func].add(tf.keras.layers.PReLU())
  models[func].add(tf.keras.layers.Dense(10))


Now we implement the model fit with different optimizers:

In [0]:
# SGD, SGD with momentum, Adam, RMSProp, Nadam
optstr = ['SGD', 'SGD with momentum', 'Adam', 'RMSProp', 'Nadam']
opt = {}
opt[0] = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0, nesterov=True)
opt[1] = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
opt[2] = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
opt[3] = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9)
opt[4] = tf.keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999)
class res:
    def __init__(self):
        self.opt = {}
        self.act = {}
        self.testloss = {}
        self.testacc = {}
        self.trainloss = {}
        self.trainacc = {}
test_res = {}
for o in range(0,5):
  print('Optimzer: '+ optstr[o])
  test_res[o] = res()
  test_res[o].opt = optstr[o]
  for model in range(0,6):
    print('ActFunc: '+ actF[model])
    test_res[o].act[model] = actF[model]
    models[model].compile(optimizer=opt[o], 
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])
    m = models[model].fit(x_train, y_train, epochs=10)
    test_res[o].trainacc[model]  =  m.history['acc']  
    test_res[o].trainloss[model]  =  m.history['loss']           
    test_res[o].testloss[model], test_res[o].testacc[model] = models[model].evaluate(x_test,  y_test, verbose=2)


Printing the output for later use:


In [0]:
for i in range(0,6):
  print('Optimzer: '+ optstr[i])
  print("Activations:")
  print(test_res[i].act)
  print("Training Accuracy")
  print(test_res[i].trainacc)
  print("Training loss")
  print(test_res[i].trainloss)
  print("Test Accuracy")
  print(test_res[i].testacc)
  print("Test loss")
  print(test_res[i].testloss)